# Parte 1: Gerar a matriz de atributos

In [1]:
# Conectando ao google Drive
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sp
from scipy.fft import fft
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split

#Endereço da base de dados no Google Drive
path_database = '/content/gdrive/MyDrive/datasets_sinais_cerebrais/'

#Autoriza a leitura do Google Drive
drive.mount('/content/gdrive')

mat = sp.loadmat(path_database+'S20.mat')             
data = np.array(mat['data'])
print(np.shape(data))

Mounted at /content/gdrive
(64, 1500, 40, 6)


In [2]:
# implementado as funçoes

def Vcar(Matriz_freq):
  media = np.mean(Matriz_freq, axis=0)
  Vcar = Matriz_freq - media
  return Vcar

def Matriz_caracteristica(Matriz_freq, array_frequencias,frequencia_observada,coleta_atual,qtd_janelas,canal): #array_frequencias[8,10,12,15]
  n_freq = len(array_frequencias)
  H_FFT = np.empty((qtd_janelas,n_freq))
  
  intervalo = int(1250/qtd_janelas)
  pontos = 0
  for janela in range(qtd_janelas):
    mag_fft = np.abs(fft(Matriz_freq[canal,pontos:pontos+intervalo]))
    pontos = pontos + intervalo
    H_FFT[janela] = mag_fft[array_frequencias]
      
    #plotar_grafico((Matriz_freq[61,pontos:pontos+250]),mag_fft,janela,frequencia_observada,coleta_atual)
     
  return H_FFT

def plotar_grafico(array_no_tempo,mag_fft,janela,frequencia_observada,coleta):
  ff = np.arange(0, 50, 0.2) # uma trial, que coresponde a 1 segundo, que corresponde a 250 pontos - 50/0.2 = 250
  print("---------------- frequencia observada: ",frequencia_observada,"coleta: ",coleta, "Janela: ",janela,"----------------")
  plt.subplot(2,1,1)
  plt.plot(array_no_tempo)
  plt.subplot(2,1,2)
  plt.plot(ff, mag_fft)
  plt.xlim(5, 15)
  plt.ylim(0, 2200)
  plt.grid()
  plt.show()

In [3]:
#gerar matriz final de atributos:

coleta = [0,1,2,3,4,5]  # de 0 a 5
freq = [0,2,4,7] # 8hz, 10hz, 12hz, 15hz
frequencias = [8,10,12,15]
canais = [47,53,54,55,56,57,60,61,62]
qtd_janelas = 5 #5 janelas -> 1segundo cada janela, 1 janela -> 5 segundos, 2 janelas -> 625pts q equivale a 2,5 segundos cada janela

for canal in canais:
  for f in freq:
    for c in coleta:
      Matriz_freq = data[:, 125:1375,f,c] #matriz gerada quando o individuo esta olhando para uma determinada frequencia
      MatrizCar = Vcar(Matriz_freq) 
      aux = Matriz_caracteristica(MatrizCar,frequencias,f,c,qtd_janelas,canal) 
      if (c ==0):
        hfft = aux
      else:
        hfft = np.vstack((hfft,aux))
    if f==0:
      hfft_2 = hfft
    else:
      hfft_2 = np.vstack((hfft_2,hfft))
  if(canal==47):
    hfft_final = hfft_2
  else:
    hfft_final = np.hstack((hfft_final,hfft_2)) 



In [4]:
print("dimensão da matriz final de atributos: ",hfft_final.shape)
print('amostra 0:')
print(hfft_final[0])

dimensão da matriz final de atributos:  (120, 36)
amostra 0:
[ 35.388066   215.26140318  85.25346112  66.03489953 401.22658118
 130.49588873 222.42061976  66.24811954 394.74002855 141.70510426
 227.31337972  68.91392904 116.53228547 233.03921678 179.85146409
  94.20172037 174.76171647  70.20318312 257.01324999 165.10241103
 201.7454916   25.13440265 224.94771343 132.12129699 467.01155916
   2.58963462 255.51055597  67.93949132 439.70997846  76.8936721
 230.61165372  97.92357418 353.25194361  51.62520197 313.01585596
 111.18088305]


# Parte 2: Classifcação

In [7]:
from sklearn.preprocessing import StandardScaler
num_linhas = len(hfft_final)
#H = np.hstack((hfft_final, np.ones((num_linhas,1))))

#one hot encoder
l1 = np.hstack((np.ones((1,30)), np.zeros((1,90))))
l2 = np.hstack((np.zeros((1,30)), np.ones((1,30)), np.zeros((1,60))))
l3 = np.hstack((np.zeros((1,60)), np.ones((1,30)), np.zeros((1,30))))
l4 = np.hstack((np.zeros((1,90)), np.ones((1,30))))

l_aux = np.vstack((l1,l2))
l_aux2 = np.vstack((l_aux,l3))
label = np.vstack((l_aux2,l4)).T

 
scaler = StandardScaler()
scaler.fit(hfft_final)
H = scaler.transform(hfft_final)
A = np.hstack((H, np.ones((num_linhas,1))))
A = A*0.01
def holdOut():
  array_acc = []
  for i in range(20):
    #particao dos dados
    X_train, X_val, y_train, y_val = train_test_split(A, label, test_size=.20, random_state=None)


    #TREINANDO O CLASSIFICADOR
    numClasses = len(frequencias)
    w = np.zeros((37, numClasses), dtype=int)

    #Calcula o vetor de pesos (hiperplano) para cada classe
    for c in range(numClasses):
        w[:,c] = np.linalg.pinv(X_train, rcond=1e-15)@y_train[:,c]

    Y = np.zeros((X_val.shape[0],numClasses))
    #VALIDANDO O CLASSIFICADOR
    for c in range(numClasses):
      Y[:,c] = np.matmul(X_val, w[:,c])
      
    maxIndexY = np.argmax(Y, axis=1) # estima do classificador
    maxIndexY_val = np.argmax(y_val, axis=1) # resposta correta

    numCorrect=0
    numWrong = 0
    for index in range(maxIndexY.shape[0]):
      if(maxIndexY[index] == maxIndexY_val[index]):
        numCorrect = numCorrect + 1
      else:
        numWrong = numWrong + 1
    acc = numCorrect/(numCorrect+numWrong)
    array_acc.append(acc)
  array_acc = np.array(array_acc)
  return array_acc.mean(), array_acc.std()

media, std = holdOut()
media, std

(0.825, 0.070464727188699)

#Parte 3: Implementação da GAN

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
import time

In [ ]:
class Gan:
  def __init__(self, epochs, noise_dim, dataset, gen_dim, lr):
    self.epochs = epochs
    self.noise_dim = noise_dim
    self.dataset = dataset
    self.gen_dim = gen_dim
    self.lr = lr
    self.cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    self.generator = self.create_generator()
    self.discriminator = self.create_discriminator()
    self.D_cycle = 10

  def create_generator(self):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_shape=(self.noise_dim,)))
    model.add(layers.Dense(128))
    model.add(layers.Dense(64))
    model.add(layers.Dense(self.gen_dim, activation="relu"))
    
    return model

  def create_discriminator(self):
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, input_shape=(self.gen_dim,)))
    model.add(layers.Dense(64))
    model.add(layers.Dense(36))
    model.add(layers.Dense(1, activation='softmax'))

    return model
    
  def D_loss(self, real_output, fake_output):
    real_loss = self.cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = self.cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
  
  def G_loss(self, fake_output):
    return self.cross_entropy(tf.ones_like(fake_output), fake_output)
  
  @tf.function
  def trainD(self, batch_samples):
    noise = tf.random.normal([batch_samples.shape[0], self.noise_dim])
    with tf.GradientTape() as disc_tape:
      generated_samples = self.generator(noise, training=True)
      real_output = self.discriminator(batch_samples, training=True)
      fake_output = self.discriminator(generated_samples, training=True)
      disc_loss = self.D_loss(real_output, fake_output)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
    self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

    return disc_loss

  @tf.function
  def trainG(self, batch_samples):
    noise = tf.random.normal([batch_samples.shape[0], self.noise_dim])
    with tf.GradientTape() as gen_tape:
      generated_samples = self.generator(noise, training=True)
      fake_output = self.discriminator(generated_samples, training=True)
      gen_loss = self.G_loss(fake_output)
    gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
    self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))

    return gen_loss

  def fit(self):
    lossG = []
    lossD = []
    self.generator_optimizer = tf.keras.optimizers.Adam(self.lr)
    self.discriminator_optimizer = tf.keras.optimizers.Adam(self.lr)
    for epoch in range(self.epochs):
      start = time.time()

      for batch_samples in self.dataset:
        Gloss = self.trainG(batch_samples)
        for _ in range(self.D_cycle):
          Dloss = self.trainD(batch_samples)
        
      lossG.append(Gloss.numpy())
      lossD.append(Dloss.numpy())
        
      print('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [ ]:
def avaliar_0_dados_reais(X_val,y_val,X_train,y_train):
  #normalização
  scaler1 = StandardScaler()
  scaler1.fit(X_train)
  X_train = scaler1.transform(X_train)
  X_val = scaler1.transform(X_val)
  

  X_train = np.hstack((X_train,np.ones((X_train.shape[0],1))))
  X_train = X_train*0.01
  X_val = np.hstack((X_val,np.ones((X_val.shape[0],1))))
  X_val = X_val*0.01

  #TREINANDO O CLASSIFICADOR
  numClasses = len(frequencias)
  w = np.zeros((37, numClasses), dtype=int)

  #Calcula o vetor de pesos (hiperplano) para cada classe
  for c in range(numClasses):
      w[:,c] = np.linalg.pinv(X_train, rcond=1e-15)@y_train[:,c]

  Y = np.zeros((X_val.shape[0],numClasses))
  #VALIDANDO O CLASSIFICADOR
  for c in range(numClasses):
    Y[:,c] = np.matmul(X_val, w[:,c])
    
  maxIndexY = np.argmax(Y, axis=1) # estima do classificador
  maxIndexY_val = np.argmax(y_val, axis=1) # resposta correta

  numCorrect=0
  numWrong = 0
  for index in range(maxIndexY.shape[0]):
    if(maxIndexY[index] == maxIndexY_val[index]):
      numCorrect = numCorrect + 1
    else:
      numWrong = numWrong + 1

  acuracia = numCorrect/(numCorrect+numWrong) 

  #matriz de confusão:
  y_true = maxIndexY_val
  y_pred = maxIndexY
  matriz_de_confusao = multilabel_confusion_matrix(y_true, y_pred)

  return acuracia, matriz_de_confusao


In [ ]:
# funçoes auxiliares para a validação
def generate_synthetic_samples(data, qtd_samples,rotulo):
  dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(data.shape[0]).batch(4)
  noise_dim = 8

  gan = Gan(epochs=150, noise_dim=noise_dim, dataset=dataset,gen_dim=36, lr=1e-4)
  gan.fit();
  noise = tf.random.normal(shape=(qtd_samples, noise_dim))
  synthetic_samples = gan.generator(noise)

  y = np.zeros((qtd_samples,4))
  y[:,rotulo] = 1.0

  return synthetic_samples.numpy(), y, gan


from sklearn.metrics import multilabel_confusion_matrix
from sklearn.preprocessing import StandardScaler
def avaliar_desempenho(X_train, y_train, X_val, y_val, X_synthetic_samples, y_synthetic_samples):

   #normalização
  scaler1 = StandardScaler()
  scaler1.fit(X_train)
  X_train = scaler1.transform(X_train)
  X_val = scaler1.transform(X_val)
  
  if (X_synthetic_samples.shape[0]!=0):
    scaler2 = StandardScaler()
    scaler2.fit(X_synthetic_samples)
    X_synthetic_samples = scaler2.transform(X_synthetic_samples)  
    X_train = np.vstack((X_train,X_synthetic_samples))
    y_train = np.vstack((y_train,y_synthetic_samples))

  X_train = np.hstack((X_train,np.ones((X_train.shape[0],1))))
  X_train = X_train*0.01
  X_val = np.hstack((X_val,np.ones((X_val.shape[0],1))))
  X_val = X_val*0.01

  #TREINANDO O CLASSIFICADOR
  numClasses = len(frequencias)
  w = np.zeros((37, numClasses), dtype=int)

  #Calcula o vetor de pesos (hiperplano) para cada classe
  for c in range(numClasses):
      w[:,c] = np.linalg.pinv(X_train, rcond=1e-15)@y_train[:,c]

  Y = np.zeros((X_val.shape[0],numClasses))
  #VALIDANDO O CLASSIFICADOR
  for c in range(numClasses):
    Y[:,c] = np.matmul(X_val, w[:,c])
    
  maxIndexY = np.argmax(Y, axis=1) # estima do classificador
  maxIndexY_val = np.argmax(y_val, axis=1) # resposta correta

  numCorrect=0
  numWrong = 0
  for index in range(maxIndexY.shape[0]):
    if(maxIndexY[index] == maxIndexY_val[index]):
      numCorrect = numCorrect + 1
    else:
      numWrong = numWrong + 1

  acuracia = numCorrect/(numCorrect+numWrong) 

  #matriz de confusão:
  y_true = maxIndexY_val
  y_pred = maxIndexY
  matriz_de_confusao = multilabel_confusion_matrix(y_true, y_pred)

  return acuracia, matriz_de_confusao


# validação

In [ ]:
l1 = np.hstack((np.ones((1,30)), np.zeros((1,90))))
l2 = np.hstack((np.zeros((1,30)), np.ones((1,30)), np.zeros((1,60))))
l3 = np.hstack((np.zeros((1,60)), np.ones((1,30)), np.zeros((1,30))))
l4 = np.hstack((np.zeros((1,90)), np.ones((1,30))))
l_aux = np.vstack((l1,l2))
l_aux2 = np.vstack((l_aux,l3))
y_real = np.vstack((l_aux2,l4)).T


def validar(n_synthetic_samples, rs):
  acuracia = []
  for i in range(20):
    y = np.zeros((30,4))
    y[:,0] = 1
    X_train_8hz, X_val_8hz, y_train_8hz, y_val_8hz = train_test_split(hfft_final[0:30,:], y, test_size=.20, random_state=rs)
    y = np.zeros((30,4))
    y[:,1] = 1
    X_train_10hz, X_val_10hz, y_train_10hz, y_val_10hz = train_test_split(hfft_final[30:60,:], y, test_size=.20, random_state=rs)
    y = np.zeros((30,4))
    y[:,2] = 1
    X_train_12hz, X_val_12hz, y_train_12hz, y_val_12hz = train_test_split(hfft_final[60:90,:], y, test_size=.20, random_state=rs)
    y = np.zeros((30,4))
    y[:,3] = 1
    X_train_15hz, X_val_15hz, y_train_15hz, y_val_15hz = train_test_split(hfft_final[90:120,:], y, test_size=.20, random_state=rs)

    indices = np.random.randint(24, size=(12))
    X_mix_bci = X_train_8hz[indices]
    y_mix_bci = y_train_8hz[indices]

    indices = np.random.randint(24, size=(12))
    X_mix_bci = np.vstack((X_mix_bci,X_train_10hz[indices]))
    y_mix_bci = np.vstack((y_mix_bci,y_train_10hz[indices]))

    indices = np.random.randint(24, size=(12))
    X_mix_bci = np.vstack((X_mix_bci,X_train_12hz[indices]))
    y_mix_bci = np.vstack((y_mix_bci,y_train_12hz[indices]))

    indices = np.random.randint(24, size=(12))
    X_mix_bci = np.vstack((X_mix_bci,X_train_15hz[indices]))
    y_mix_bci = np.vstack((y_mix_bci,y_train_15hz[indices]))

    X_train_bci = np.vstack((X_train_8hz,X_train_10hz))
    X_train_bci = np.vstack((X_train_bci,X_train_12hz))
    X_train_bci = np.vstack((X_train_bci,X_train_15hz))

    y_train_bci = np.vstack((y_train_8hz,y_train_10hz))
    y_train_bci = np.vstack((y_train_bci,y_train_12hz))
    y_train_bci = np.vstack((y_train_bci,y_train_15hz))

    X_val_bci = np.vstack((X_val_8hz,X_val_10hz))
    X_val_bci = np.vstack((X_val_bci,X_val_12hz))
    X_val_bci = np.vstack((X_val_bci,X_val_15hz))

    y_val_bci = np.vstack((y_val_8hz,y_val_10hz))
    y_val_bci = np.vstack((y_val_bci,y_val_12hz))
    y_val_bci = np.vstack((y_val_bci,y_val_15hz))


    synthetic_samples_8hz, y_8hz, gan_8hz = generate_synthetic_samples(X_train_8hz, n_synthetic_samples,0)
    synthetic_samples_10hz, y_10hz, gan_10hz = generate_synthetic_samples(X_train_10hz, n_synthetic_samples,1)
    synthetic_samples_12hz, y_12hz, gan_12hz = generate_synthetic_samples(X_train_12hz, n_synthetic_samples,2)
    synthetic_samples_15hz, y_15hz, gan_15hz = generate_synthetic_samples(X_train_15hz, n_synthetic_samples,3)

    X_synthetic_samples = np.vstack((synthetic_samples_8hz,synthetic_samples_10hz))
    X_synthetic_samples = np.vstack((X_synthetic_samples,synthetic_samples_12hz))
    X_synthetic_samples = np.vstack((X_synthetic_samples,synthetic_samples_15hz))

    y_synthetic_samples = np.vstack((y_8hz,y_10hz))
    y_synthetic_samples = np.vstack((y_synthetic_samples,y_12hz))
    y_synthetic_samples = np.vstack((y_synthetic_samples,y_15hz))

    

    X_mix_bci = np.vstack((X_mix_bci,X_synthetic_samples))
    y_mix_bci = np.vstack((y_mix_bci,y_synthetic_samples))


    ac,mc = avaliar_desempenho(X_train_bci, y_train_bci, X_val_bci, y_val_bci, X_synthetic_samples, y_synthetic_samples)
    #ac,mc = avaliar_0_dados_reais(X_val_bci,y_val_bci,X_synthetic_samples,y_synthetic_samples)
    acuracia.append(ac)
  acuracia = np.array(acuracia)
  return acuracia.mean(),acuracia.std(),acuracia

In [ ]:
testar = [4,6]
array_media = []
array_std = []
array_acc = []
inicio = time.time()
for x in testar:
  media,std,acc = validar(x, None)
  array_media.append(media)
  array_std.append(std)
  array_acc.append(acc)
fim = time.time()
tempo_gasto = fim - inicio


/usr/local/lib/python3.8/dist-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Time for epoch 1 is 3.941063642501831 sec
Time for epoch 2 is 0.14205360412597656 sec
Time for epoch 3 is 0.1429135799407959 sec
Time for epoch 4 is 0.10698986053466797 sec
Time for epoch 5 is 0.1422569751739502 sec
Time for epoch 6 is 0.1430051326751709 sec
Time for epoch 7 is 0.1430799961090088 sec
Time for epoch 8 is 0.14325261116027832 sec
Time for epoch 9 is 0.1268167495727539 sec
Time for epoch 10 is 0.1431126594543457 sec
Time for epoch 11 is 0.1427624225616455 sec
Time for epoch 12 is 0.14317727088928223 sec
Time for epoch 13 is 0.14308738708496094 sec
Time for epoch 14 is 0.14180397987365723 sec
Time for epoch 15 is 0.1437549591064453 sec
Time for epoch 16 is 0.1093602180480957 sec
Time for epoch 17 is 0.13640832901000977 sec
Time for epoch 18 is 0.1422581672668457 sec
Time for epoch 19 is 0.14297938346862793 sec
Time for epoch 20 is 0.14147257804870605 sec
Time for epoch 21 is 0.14406633377075195 sec
Time for epoch 22 is 0.10395932197570801 sec
Time for epoch 23 is 0.14331865

/usr/local/lib/python3.8/dist-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Time for epoch 1 is 2.689455509185791 sec
Time for epoch 2 is 0.14314794540405273 sec
Time for epoch 3 is 0.09635114669799805 sec
Time for epoch 4 is 0.08939814567565918 sec
Time for epoch 5 is 0.14179658889770508 sec
Time for epoch 6 is 0.1424260139465332 sec
Time for epoch 7 is 0.14339995384216309 sec
Time for epoch 8 is 0.14305520057678223 sec
Time for epoch 9 is 0.1418321132659912 sec
Time for epoch 10 is 0.14300799369812012 sec
Time for epoch 11 is 0.10138273239135742 sec
Time for epoch 12 is 0.14096975326538086 sec
Time for epoch 13 is 0.10006427764892578 sec
Time for epoch 14 is 0.1421976089477539 sec
Time for epoch 15 is 0.14417409896850586 sec
Time for epoch 16 is 0.14173293113708496 sec
Time for epoch 17 is 0.1426537036895752 sec
Time for epoch 18 is 0.1428818702697754 sec
Time for epoch 19 is 0.14232349395751953 sec
Time for epoch 20 is 0.09650492668151855 sec
Time for epoch 21 is 0.1425313949584961 sec
Time for epoch 22 is 0.14183616638183594 sec
Time for epoch 23 is 0.1424

/usr/local/lib/python3.8/dist-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


...

Time for epoch 1 is 2.9320034980773926 sec
Time for epoch 2 is 0.14316534996032715 sec
Time for epoch 3 is 0.14627909660339355 sec
Time for epoch 4 is 0.10085773468017578 sec
Time for epoch 5 is 0.09666728973388672 sec
Time for epoch 6 is 0.14281940460205078 sec
Time for epoch 7 is 0.1418614387512207 sec
Time for epoch 8 is 0.14263916015625 sec
Time for epoch 9 is 0.1421365737915039 sec
Time for epoch 10 is 0.14672040939331055 sec
Time for epoch 11 is 0.14222288131713867 sec
Time for epoch 12 is 0.14770150184631348 sec
Time for epoch 13 is 0.30432796478271484 sec
Time for epoch 14 is 0.30479907989501953 sec
Time for epoch 15 is 0.30173230171203613 sec
Time for epoch 16 is 0.30411410331726074 sec
Time for epoch 17 is 0.3040792942047119 sec
Time for epoch 18 is 0.30608344078063965 sec
Time for epoch 19 is 0.30292654037475586 sec
Time for epoch 20 is 0.16507744789123535 sec
Time for epoch 21 is 0.303804874420166 sec
Time for epoch 22 is 0.3038642406463623 sec
Time for epoch 23 is 0.152116

/usr/local/lib/python3.8/dist-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Time for epoch 1 is 5.10897421836853 sec
Time for epoch 2 is 0.30298542976379395 sec
Time for epoch 3 is 0.3023507595062256 sec
Time for epoch 4 is 0.304795503616333 sec
Time for epoch 5 is 0.3036196231842041 sec
Time for epoch 6 is 0.1425459384918213 sec
Time for epoch 7 is 0.146622896194458 sec
Time for epoch 8 is 0.14309453964233398 sec
Time for epoch 9 is 0.14659452438354492 sec
Time for epoch 10 is 0.09690070152282715 sec
Time for epoch 11 is 0.142486572265625 sec
Time for epoch 12 is 0.10635924339294434 sec
Time for epoch 13 is 0.14253473281860352 sec
Time for epoch 14 is 0.09324026107788086 sec
Time for epoch 15 is 0.14363789558410645 sec
Time for epoch 16 is 0.10088086128234863 sec
Time for epoch 17 is 0.14176702499389648 sec
Time for epoch 18 is 0.14278769493103027 sec
Time for epoch 19 is 0.14954495429992676 sec
Time for epoch 20 is 0.12910866737365723 sec
Time for epoch 21 is 0.11330747604370117 sec
Time for epoch 22 is 0.1413898468017578 sec
Time for epoch 23 is 0.117205619

/usr/local/lib/python3.8/dist-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Time for epoch 1 is 3.142324924468994 sec
Time for epoch 2 is 0.30512523651123047 sec
Time for epoch 3 is 0.14259839057922363 sec
Time for epoch 4 is 0.30179667472839355 sec
Time for epoch 5 is 0.1460108757019043 sec
Time for epoch 6 is 0.30315232276916504 sec
Time for epoch 7 is 0.30414509773254395 sec
Time for epoch 8 is 0.14205574989318848 sec
Time for epoch 9 is 0.3041188716888428 sec
Time for epoch 10 is 0.14101076126098633 sec
Time for epoch 11 is 0.301555871963501 sec
Time for epoch 12 is 0.30576348304748535 sec
Time for epoch 13 is 0.1423501968383789 sec
Time for epoch 14 is 0.14339447021484375 sec
Time for epoch 15 is 0.14197802543640137 sec
Time for epoch 16 is 0.14212989807128906 sec
Time for epoch 17 is 0.08864927291870117 sec
Time for epoch 18 is 0.14118742942810059 sec
Time for epoch 19 is 0.10097861289978027 sec
Time for epoch 20 is 0.14298486709594727 sec
Time for epoch 21 is 0.09817004203796387 sec
Time for epoch 22 is 0.1426987648010254 sec
Time for epoch 23 is 0.0930

In [ ]:
print('tempo: ',(tempo_gasto/60),' min' )
df = pd.DataFrame(testar, columns = ['amostras fake por classe'])
df['acc-media'] = array_media
df['acc-desv.pad.'] = array_std
df

tempo:  70.20661771694819  min


,amostras fake por classe,acc-media,acc-desv.pad.
0,4,0.816667,0.079495
1,6,0.810417,0.076234


. zero dados fake, 24 dados reais por classe: (0.8216666666666669, 0.051934788169609614) rs = none

. 12 dados fake por classe, 12 dados reais por classe: (0.53125	0.098491) rs = none

. 24 dados fake por classe, 0 dados reais: ( )

.1000 dados fake por classe, 0 dados reais: ( )